# Exercise 01: Running Sumo Simulations

This tutorial walks through the process of running non-RL traffic simulations in Flow. Simulations of this form act as non-autonomous baselines and depict the behavior of human dynamics on a network. Similar simulations may also be used to evaluate the performance of hand-designed controllers on a network. This tutorial focuses primarily on the former use case, while an example of the latter may be found in `exercise07_controllers.ipynb`.

In this exercise, we simulate a initially perturbed single lane ring road. We witness in simulation that as time advances the initially perturbations do not dissipate, but instead propagates and expands until vehicles are forced to periodically stop and accelerate. For more information on this behavior, we refer the reader to the following article [1].

## 1. Components of a Simulation
All simulations, both in the presence and absence of RL, require two components: a *scenario*, and an *environment*. Scenarios describe the features of the transportation network used in simulation. This includes the positions and properties of nodes and edges constituting the lanes and junctions, as well as properties of the vehicles, traffic lights, inflows, etc. in the network. Environments, on the other hand, initialize, reset, and advance simulations, and act the primary interface between the reinforcement learning algorithm and the scenario. Moreover, custom environments may be used to modify the dynamical features of an scenario.

## 2. Setting up a Scenario
Flow contains a plethora of pre-designed scenarios used to replicate highways, intersections, and merges in both closed and open settings. All these scenarios are located in flow/scenarios. In order to recreate a ring road network, we begin by importing the scenario `LoopScenario`.

In [1]:
from flow.scenarios.loop.loop_scenario import LoopScenario

This scenario, as well as all other scenarios in Flow, is parametrized by the following arguments: 
* name
* generator_class
* vehicles
* net_params
* initial_config
* traffic_lights

These parameters allow a single scenario to be recycled for a multitude of different network settings. For example, `LoopScenario` may be used to create ring roads of variable length with a variable number of lanes and vehicles.

### 2.1 Name
The `name` argument is a string variable depicting the name of the scenario. This has no effect on the type of network created.

In [2]:
name = "ring_example"

### 2.2 Generator Class
Generator classes are used to create configuration and net files needed to initialize a simulation instance. The methods of this class are called by the base scenario class. All scenarios in Flow are come with complementary generators located in the same directory as the scenario. In the case of `LoopScenario`, this generator is called `CircleGenerator`.

In [3]:
from flow.scenarios.loop.gen import CircleGenerator

### 2.3 Vehicles
The `Vehicles` class stores state information on all vehicles in the network. This class is used to identify the dynamical behavior of a vehicle and whether it is controlled by a reinforcement learning agent. Morover, information pertaining to the observations and reward function can be collected from various get methods within this class.

The initial configuration of this class describes the number of vehicles in the network at the start of every simulation, as well as the properties of these vehicles. We begin by creating an empty `Vehicles` object.

In [4]:
from flow.core.vehicles import Vehicles

vehicles = Vehicles()

Once this object is created, vehicles may be introduced using the `add` method. This method specifies the types and quantities of vehicles at the start of a simulation rollout. For a description of the various arguements associated with the `add` method, we refer the reader to the following documentation (reference readthedocs).

When adding vehicles, their dynamical behaviors may be specified either by the simulator (default), or by user-generated models. For longitudinal (acceleration) dynamics, several prominent car-following models are implemented in Flow. For this example, the acceleration behavior of all vehicles will be defined by the Intelligent Driver Model (IDM) [2].

In [5]:
from flow.controllers.car_following_models import IDMController

Another controller we define is for the vehicle's routing behavior. For closed network where the route for any vehicle is repeated, the `ContinuousRouter` controller is used to perpetually reroute all vehicles to the initial set route.

In [6]:
from flow.controllers.routing_controllers import ContinuousRouter

Finally, we add 22 vehicles of type "human" with the above acceleration and routing behavior into the `Vehicles` class.

In [18]:
dir(vehicles)

['_Vehicles__controlled_ids',
 '_Vehicles__controlled_lc_ids',
 '_Vehicles__human_ids',
 '_Vehicles__ids',
 '_Vehicles__observed_ids',
 '_Vehicles__rl_ids',
 '_Vehicles__sumo_obs',
 '_Vehicles__vehicles',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_departed',
 '_ids_by_edge',
 '_multi_lane_headways',
 '_multi_lane_headways_util',
 '_next_edge_leaders',
 '_num_arrived',
 '_num_departed',
 '_prev_edge_followers',
 'add',
 'get_absolute_position',
 'get_acc_controller',
 'get_controlled_ids',
 'get_controlled_lc_ids',
 'get_edge',
 'get_follower',
 'get_headway',
 'get_human_ids',
 'get_ids',
 'get_ids_by_edge',
 'get_inflow_rate',
 'get_initial_speed',
 'get_lane',

In [7]:
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=22)

### 2.4 NetParams

`NetParams` are network-specific parameters used to define the shape and properties of a network. Unlike most other parameters, `NetParams` may vary drastically depending on the specific network configuration, and accordingly most of its parameters are stored in `additional_params`. In order to determine which `additional_params` variables may be needed for a specific scenario, we refer to the `ADDITIONAL_NET_PARAMS` variable located in the scenario file.

In [8]:
from flow.scenarios.loop.loop_scenario import ADDITIONAL_NET_PARAMS

print(ADDITIONAL_NET_PARAMS)

{'lanes': 1, 'length': 230, 'speed_limit': 30, 'resolution': 40}


Importing the `ADDITIONAL_NET_PARAMS` dict from the ring road scenario, we see that the required parameters are:

* **length**: length of the ring road
* **lanes**: number of lanes
* **speed**: speed limit for all edges
* **resolution**: resolution of the curves on the ring. Setting this value to 1 converts the ring to a diamond.


At times, other inputs (for example `no_internal_links`) may be needed from `NetParams` to recreate proper network features/behavior. These requirements can be founded in the scenario's documentation. For the ring road, no attributes are needed aside from the `additional_params` terms. Furthermore, for this exercise, we use the scenario's default parameters when creating the `NetParams` object.

In [9]:
from flow.core.params import NetParams

net_params = NetParams(additional_params=ADDITIONAL_NET_PARAMS)

### 2.5 InitialConfig

`InitialConfig` specifies parameters that affect the positioning of vehicle in the network at the start of a simulation. These parameters can be used to limit the edges and number of lanes vehicles originally occupy, and provide a means of adding randomness to the starting positions of vehicles. In order to introduce a small initial disturbance to the system of vehicles in the network, we set the `perturbation` term in `InitialConfig` to 1m.

In [10]:
from flow.core.params import InitialConfig

initial_config = InitialConfig(spacing="uniform", perturbation=1)

### 2.6 TrafficLights

`TrafficLights` are used to desribe the positions and types of traffic lights in the network. These inputs are outside the scope of this tutorial, and instead are covered in `exercise06_traffic_lights.ipynb`. For our example, we create an empty `TrafficLights` object, thereby ensuring that none are placed on any nodes.

In [11]:
from flow.core.traffic_lights import TrafficLights

traffic_lights = TrafficLights()

## 3. Setting up an Environment

Several envionrments in Flow exist to train autonomous agents of different forms (e.g. autonomous vehicles, traffic lights) to perform a variety of different tasks. These environments are often scenario or task specific; however, some can be deployed on an ambiguous set of scenarios as well. One such environment, `AccelEnv`, may be used to train a variable number of vehicles in a fully observable network with a *static* number of vehicles.

In [12]:
from flow.envs.loop.loop_accel import AccelEnv

Although we will not be training any autonomous agents in this exercise, the use of an environment allows us to view the cumulative reward simulation rollouts receive in the absence of autonomy.

Envrionments in Flow are parametrized by three components:
* `EnvParams`
* `SumoParams`
* `Scenario`

### 3.1 SumoParams
`SumoParams` specifies simulation-specific variables. These variables include the length a simulation step (in seconds) and whether to render the GUI when running the experiment. For this example, we consider a simulation step length of 0.1s and activate the GUI.

In [13]:
from flow.core.params import SumoParams

sumo_params = SumoParams(sim_step=0.1, render=True)

### 3.2 EnvParams

`EnvParams` specify environment and experiment-specific parameters that either affect the training process or the dynamics of various components within the scenario. Much like `NetParams`, the attributes associated with this parameter are mostly environment specific, and can be found in the environment's `ADDITIONAL_ENV_PARAMS` dictionary.

In [14]:
from flow.envs.loop.loop_accel import ADDITIONAL_ENV_PARAMS

print(ADDITIONAL_ENV_PARAMS)

{'max_accel': 3, 'max_decel': 3, 'target_velocity': 10}


Importing the `ADDITIONAL_ENV_PARAMS` variable, we see that it consists of only one entry, "target_velocity", which is used when computing the reward function associated with the environment. We use this default value when generating the `EnvParams` object.

In [15]:
from flow.core.params import EnvParams

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

## 4. Setting up and Running the Experiment
Once the inputs to the scenario and environment classes are ready, we are ready to set up a `SumoExperiment` object.

In [16]:
from flow.core.experiment import SumoExperiment

These objects may be used to simulate rollouts in the absence of reinforcement learning agents, as well as acquire behaviors and rewards that may be used as a baseline with which to compare the performance of the learning agent. In this case, we choose to run our experiment for one rollout consisting of 3000 steps (300 s).

**Note**: When executing the below code, remeber to click on the    <img style="display:inline;" src="img/play_button.png"> Play button after the GUI is rendered.

In [17]:
# create the scenario object
scenario = LoopScenario(name="ring_example",
                        generator_class=CircleGenerator,
                        vehicles=vehicles,
                        net_params=net_params,
                        initial_config=initial_config,
                        traffic_lights=traffic_lights)

# create the environment object
env = AccelEnv(env_params, sumo_params, scenario)

# create the experiment object
exp = SumoExperiment(env, scenario)

# run the experiment for a set number of rollouts / time steps
exp.run(1, 3000)

Round 0, return: 796.1684998053887
Average, std return: 796.1684998053887, 0.0
Average, std speed: 2.9223870365326996, 0.0


{'mean_returns': [0.2653894999351284],
 'per_step_returns': [[0.008153291356272878,
   0.01618296356095252,
   0.02409023939994074,
   0.03187653489080189,
   0.039543334559643296,
   0.04709208413493783,
   0.05452410367705526,
   0.06184052214802495,
   0.06904223283224631,
   0.07612986790747504,
   0.08310378980727028,
   0.0899640967284633,
   0.09671063961728073,
   0.10334304811648481,
   0.10986076319157767,
   0.11626307441761773,
   0.12254916016255933,
   0.1287181291300908,
   0.1347690619206315,
   0.1407010514384831,
   0.14651324112593167,
   0.15220486015207066,
   0.15777525483423466,
   0.16322391572850103,
   0.1685504999936783,
   0.17375484880723205,
   0.17883699978513481,
   0.1837971945222117,
   0.1886358815166027,
   0.1933537148641656,
   0.19795154920113273,
   0.20243043143414968,
   0.20679158982692125,
   0.2110364210153617,
   0.21516647550326864,
   0.21918344215363242,
   0.22308913214221016,
   0.22688546278467409,
   0.23057444159029927,
   0.2341581

As we can see from the above simulation, the initial perturbations in the network instabilities propogate and intensify, eventually leading to the formation of stop-and-go waves after approximately 180s.

## 5. Modifying the Simulation
This tutorial has walked you through running a single lane ring road experiment in Flow. As we have mentioned before, these simulations are highly parametrizable. This allows us to try different representations of the task. For example, what happens if no initial perturbations are introduced to the system of homogenous human-driven vehicles?

```
initial_config = InitialConfig()
```

In addition, how does the task change in the presence of multiple lanes where vehicles can overtake one another?

```
net_params = NetParams(
    additional_params={
        'length': 230, 
        'lanes': 2, 
        'speed_limit': 30, 
        'resolution': 40
    }
)
```

Feel free to experiment with all these problems and more!

## Bibliography
[1] Sugiyama, Yuki, et al. "Traffic jams without bottlenecks—experimental evidence for the physical mechanism of the formation of a jam." New journal of physics 10.3 (2008): 033001.

[2] Treiber, Martin, Ansgar Hennecke, and Dirk Helbing. "Congested traffic states in empirical observations and microscopic simulations." Physical review E 62.2 (2000): 1805.